In [ ]:
import glob

files = glob.glob('img/*.jpg')

In [ ]:
import numpy as np
from keras.preprocessing.image import load_img, img_to_array

imgs = np.array([img_to_array(load_img(f)) for f in files])

imgs.shape

In [ ]:
import os

th = 128

labels = np.array([img_to_array(load_img(f"{os.path.splitext(f)[0]}_edge.png", color_mode = 'grayscale')) for f in files])

labels[labels < th] = 0
labels[labels >= th] = 1

labels.shape


In [ ]:
b_len = len(labels[labels == 0])
w_len = len(labels[labels == 1])

print(f"0 = {b_len}, 1 = {w_len}")

b_len / w_len

In [ ]:
from keras.models import Sequential
from keras.layers import Dropout, UpSampling2D, Activation
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.layers.normalization import BatchNormalization

model = Sequential()

model.add(BatchNormalization(input_shape = imgs.shape[1:]))
          
model.add(Conv2D(16, 3, padding='same', activation = 'relu'))
model.add(MaxPool2D())

model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(32, 3, padding='same', activation = 'relu'))
model.add(Conv2D(32, 3, padding='same', activation = 'relu'))
model.add(MaxPool2D())

model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(64, 3, padding='same', activation = 'relu'))
model.add(Conv2D(64, 3, padding='same', activation = 'relu'))
model.add(MaxPool2D())

model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(128, 3, padding='same', activation = 'relu'))

model.add(UpSampling2D())
model.add(Conv2D(64, 3, padding='same', activation = 'relu'))
model.add(Conv2D(64, 3, padding='same', activation = 'relu'))

model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(UpSampling2D())
model.add(Conv2D(32, 3, padding='same', activation = 'relu'))
model.add(Conv2D(32, 3, padding='same', activation = 'relu'))

model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(UpSampling2D())
model.add(Conv2D(16, 3, padding='same', activation = 'relu'))

model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(1, 1, padding='valid'))

model.add(Activation('sigmoid'))

model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['mae'])

model.summary()

In [ ]:
hist = model.fit(imgs, labels, epochs = 5, batch_size = 10)

hist

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

plt.plot(hist.history['loss'])

In [ ]:
model.save('model/cnn_encoder-decoder1.hd5')

In [ ]:
def predict(index, s = 14.0):
    plt.rcParams['figure.figsize'] = (s, s)

    size = imgs.shape[1:-1]
    
    pred = model.predict(np.array([imgs[index]]))[0]
    pred = pred * 255

    plt.subplot(1, 3, 1)
    plt.imshow(imgs[index].astype(int))

    plt.subplot(1, 3, 2)
    plt.imshow(labels[index].reshape(size), cmap = 'gray')

    plt.subplot(1, 3, 3)
    plt.imshow(pred.reshape(size), cmap = 'gray')

In [ ]:
predict(0)

In [ ]:
predict(1)

In [ ]:
predict(2)